# Robotics Foundations (H) - Lab 3

## ILOs

At the end of the lab, you should be able to:
* Use ROS’ motion planning framework, MoveIt
* Learn about different planning algorithms via the Open Motion Planning Library (OMPL)
* Use MoveIt Python API to move Baxter


## Introduction
### MoveIt!

**NOTE: ** If you attended Lecture 5 "Understanding paths and motions", skim through this section and start with "MoveIt! in Action" below.

MoveIt! is a set of tools for robotic manipulation in ROS. The [main web page](http://moveit.ros.org) contains documentation, tutorials, and installation instructions as well as example demonstrations with several [robotic arms (or robots)](http://moveit.ros.org/robots/) that use MoveIt!. These robots include Baxter, Atlas (used by Darpa) and Robonaut (an astronaut robot developed by NASA); all of them free through Gazebo and can be controlled with MoveIt!

The library incorporates fast and off-the-shelf inverse kinematics solvers (as part of the motion planning primitives), state-of-the-art algorithms for manipulation, grasping 3D perception (usually in the form of point clouds), kinematics, control, and navigation. The architecture of MoveIt! is shown in the following diagram taken from http://moveit.ros.org/documentation/concepts/. Feel free to read the documentation if you wish, but it is not needed for RF.

![moveit_overview.jpg](imgs/moveit_overview.jpg)
<div style="text-align:center"><b>Figure 1: MoveIt! System Architecture</b><br></div>

### Motion Planning and Kinematics
Motion planning deals with the problem of moving the arm to a configuration, allowing you to reach a pose with the end effector without crashing with an obstacle. The MoveIt! user interface will enable you to use different libraries for motion planning, such as OMPL (http://ompl.kavrakilab.org ), using [ROS actions](http://wiki.ros.org/actionlib), or services. ROS actions are similar to services, but actions can take a long time to execute, and you might want to have the ability to cancel a request during execution or get periodic feedback about the progress of the action.

In ROS, the pipeline for motion planning is as follows:

* A planning request is sent to the motion planning module, which takes care of avoiding collisions (including self-collisions) and finds a trajectory for all the joints that move the arm, so it reaches the goal requested. 
* Such goal consists of a location in joint space or an end effector pose, which could include an object as well as kinematic constraints, such as position, orientation, and user-specified constraints.
* The result of the motion plan is a trajectory that moves the arm to the target goal location. This trajectory also avoids collisions and satisfies the velocity and acceleration constraints at the joint level, so the arm moves smoothly.

Forward and Inverse Kinematics are integrated within MoveIt! (i.e. `RobotState` class). For inverse kinematics, MoveIt! provides a default plugin that uses a numerical Jacobian-based solver that is automatically configured by the Setup Assistant - the basis of this assistant is to provide an easy-to-use UI to enable MoveIt! for any robot (you can watch a video tutorial about the assistant here: https://www.youtube.com/watch?v=_5iUDyW3a3A; although it is not required to progress with this lab). As we went over the lecture, you can design a robot and integrate it within MoveIt! given your knowledge of URDF and XACRO! You can also write your inverse kinematic plugin if needed but the default numerical solvers usually are enough.

## MoveIt! in Action

The most simple way to see MoveIt! in action is using its RViz plugin. To do this, open three terminals and do the following, each command in different terminals and press enter sequentially:

Terminal 1:
``` bash
roslaunch baxter_gazebo baxter_world.launch
```

**Note: Wait until Gazebo is up (e.g. when you see a message in the terminal stating "Robot is disabled") to run the following commands.**

Terminal 2 (one command at a time):
``` bash
rosrun baxter_tools enable_robot.py -e
rosrun baxter_interface joint_trajectory_action_server.py
```

Terminal 3:
``` bash
roslaunch baxter_moveit_config baxter_grippers.launch
```

**NOTE: We recommend you maximise the RViz window. You can move it to a different Desktop by dragging the RViz window to the right of the desktop until you change to a new Desktop. You can naviagte between Desktops by pressing Ctrl+Alt+Left or Right**

`enable_robot.py` tells Baxter to be ready to accept commands; `joint_trajectory_action_server.py` is the controller of (i.e. interface to) the robot. The trajectory action server takes a trajectory as a ROS action goal and executes it. `baxter_grippers.launch` is a launch file that starts RViz with MoveIt! plugin. This launch file is part of `baxter_moveit_config` package which you can get from the official [ROS MoveIt! robots package](https://github.com/ros-planning/moveit_robots). This package is already installed in your `ros_ws` in the virtual machine but if you wish to get it in your PC, clone the repository in `~/ros_ws/src` and then `catkin_make & catkin_make install`. By now, you should have Baxter in RViz, as shown below:

![baxter_moveitrviz.png](imgs/baxter_moveitrviz.png)
<div style="text-align:center"><b>Figure 2: RViz and MoveIt!</b><br></div>

You start playing with the interactive markers on Baxter’s hands and watch the arm move. To do this, press "Interact" in the top menu of Rviz; this should be the default. You should see interactive markers appear on the arms - you can interact by clicking the arrows, wheels or sphere; you should see the arm moving as you move your mouse cursor.

Now, find the large panel at the bottom left of the screen – this is the MoveIt Rviz Motion Planning Plugin that allows you to test different motion plans. It should have the words OMPL in green – indicating that you are connected to a planning library. You should see something like this:

![moveit_plugin.png](imgs/moveit_plugin.png)
<div style="text-align:center"><b>Figure 3: MoveIt! plugin enabled</b><br></div>

With the MoveIt RViz Motion Planning Plugin ("MotionPlanning" pane in Figure 4); let's do the following:

* Choose an algorithm from the drop-down menu at he bottom of the green “OMPL” text to specify what type of algorithm you want to use. You can leave it as "RRTConnectkConfigDefault" for the moment.
* Start configuring the plugin for the Baxter. Expand the “MotionPlanning” section in “Displays.”
* Under the Planning Request section, change the "Planning Group" to different groups to see how you can control different parts of the robot

![boxes.png](imgs/boxes.png)
<div style="text-align:center"><b>Figure 4: RViz panels</b><br></div>

There are four different visualisations of the robot available (the display states for these visualisations, can be toggled on and off using the checkboxes):

* The "Query Start State" for motion planning in the “Planning Request” (represented in green).
* The "Query Goal State" for motion planning in the “Planning Request” (represented in orange).

These visualisations are helpful while debugging robot trajectories. Now it is time to interact with Baxter, so:

* Press Interact in the top menu of Rviz.
* One marker (corresponding to the orange coloured right arm) will be used to set the “Goal State” for motion planning. Another marker corresponding to a green coloured representation of the right arm will be used to set the “Start State” for motion planning. You will be able to use these markers (which are attached to the tip link of each arm) to drag the arm around and change its orientation.
* Note what happens when you try to move the arm into collision with itself or the workspace. The two links that are in collision will turn red. The “Use Collision-Aware IK” checkbox in the Motion Planning pane (box below Displays pnae) on the Planning tab (see figure 4) allows you to toggle the behaviour of the IK solver. When the checkbox is ticked, the solver will keep attempting to find a collision-free solution for the desired end-effector pose. When it is unticked, the solver will allow collisions to happen in the solution. The links in collision will always still be visualised in red, regardless of the state of the checkbox.
* Note also what happens when you try to move the end-effector out of its reachable workspace.

After setting a "Goal State" using the interactive markers, you can now plan a motion! To do this, 

* Make sure that the goal state is not in collision with the robot itself and make sure the "Show Robot Visual" in Planned Path is being visualised.
* In the Planning tab (bottom left tab), press the Plan button. You should be able to see a visualisation of the arm moving.
* Try changing the algorithm being used to solve the motion planning problem (Context tab)

Finally, press "Execute" in the Planning tab to move the robot! You should see Baxter moving in the simulator. We encourage you to try all the available planning algorithms in the Context tab; it is usual to see the planning animation making Tai-Chi like movements. Robot motion planning is an open research problem, and a solution that works efficiently has yet to be developed!

## Using MoveIt in Python

To start with, let's import the required libraries (press Ctrl-Enter / Shift-Enter the following cell)

In [ ]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg
from std_msgs.msg import String

Now, we need to setup `moveit_commander` and init the ROS node. This notebook is serving as ROS node; this is convenient when you want to prototype ROS nodes/scripts.

In [ ]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('lab3', anonymous=True)

After that, we can now instantiate the different objects required to control Baxter. 

In [ ]:
# The "RobotCommander" object is an interface to Baxter (or any robot) as a whole.
robot = moveit_commander.RobotCommander()

# This is an interface to the world surrounding the robot.
scene = moveit_commander.PlanningSceneInterface()

# This is an interface to one group of joints.  In our case, we want to use the "right_arm".
#We will use this to plan and execute motions
group = moveit_commander.MoveGroupCommander("left_arm")

Now, let's explore what MoveIt! has to offer, for instance:

In [ ]:
# This will give you the frame that the robot is attached, for Baxter, this is the "world" frame
group.get_planning_frame()

In [ ]:
# It is also possible to find out what the end-effector of the robot
group.get_end_effector_link()

Do check out the MoveIt! [Python API](http://docs.ros.org/kinetic/api/moveit_commander/html/), some functions might come handy during your assessed excercise! For now, let's focus on getting Baxter to move. First, you need to plan for a pose goal (if it fails for some reason, e.g. no plan, just re-run the cell again):

In [ ]:
pose_target = geometry_msgs.msg.Pose()
# 3D point and quaternion (same as previous lab, e.g. 0.644, 0.0, 0.0 for XYZ and
# -0.381, 0.923, -0.015, 0.052 for the XYZW components of the quaternion).
# NOTE: This point might fail if it is close or in the box! Choose a different point if that's the case

# pose_target is initialised with zeros in all entries so let's popualate the quaternion
pose_target.orientation.x = -0.381
pose_target.orientation.y = 0.923
pose_target.orientation.z = -0.015
pose_target.orientation.w = 0.052


pose_target.position.x = 0.644
pose_target.position.y = 0.0
pose_target.position.z = 0.0

# Now, add your Pose msg to the group's pose target
group.set_pose_target(pose_target)

# and compute the plan!
plan = group.plan()
print(plan)

If you missed the plan in RViz, check the box for "Loop Animation" in the Planned Path list, Displays pane. The computed plan will be from the starting pose (i.e. the current state of the robot) to the goal pose we have set above. To execute the plan, run the following (keep an eye on RViz and Gazebo to see the actual execution!):

In [ ]:
ret = group.execute(plan)

and voilà! We got Baxter to move one of its arms! To further your understanding, explore different 3D points and orientations by changing the Pose values above.

You can now make Baxter move along a set of points using MoveIt! For this, you will use the "[Cartesian Paths](http://moveit.ros.org/moveit!/ros/2015/01/29/new-capabilities-in-moveit-the-cartesian-path-planner-plugin.html)" capabilities of MoveIt! These allow you to plan a path by specifying a list of waypoints for the end-effector to go through. To demonstrate the latter, let's define 3 waypoints, plan and then execute:

In [ ]:
waypoints = []

# start with the current pose
waypoints.append(group.get_current_pose().pose)

# first orient gripper and move forward (+x)
wpose = geometry_msgs.msg.Pose()
wpose.orientation.w = 1.0
wpose.position.x = waypoints[0].position.x + 0.1
wpose.position.y = waypoints[0].position.y
wpose.position.z = waypoints[0].position.z
waypoints.append(copy.deepcopy(wpose))

# second move down
wpose.position.z -= 0.10
waypoints.append(copy.deepcopy(wpose))

# third move to the side
wpose.position.y += 0.05
waypoints.append(copy.deepcopy(wpose))

# The cartesian path will be interpolated at a resolution of 1 cm (i.e. 0.01 as the second argument below
# for the end-effector. The third argument is to disable the "jump threshold" see:
# http://docs.ros.org/kinetic/api/moveit_commander/html/classmoveit__commander_1_1move__group_1_1MoveGroupCommander.html#a4a3cfd21dd94bcc6991797e474c4d7f3
(plan_cartesian, fraction) = group.compute_cartesian_path(waypoints, 0.01, 0.0)

# If everything worked out fine, execute the above plan!
group.execute(plan_cartesian)

### Challenge!

Can you make Baxter say hello using the above code? Repurpose the code above and define more waypoints to get Baxter to wave one of its arms. You can also create a ROS package and create a new ROS node if you do not want to use jupyter. You can now command Baxter to whatever you want! If you create a ROS package, remember to create a new package at `~/rf_ws/src`, run `catkin_make` and source your RF workspace before running your node!

**NOTE: This is not an assessed exercise, this is to improve your learning and understanding of MoveIt! However, we suggest you giving it a try as this will serve as the basis for the assessed excercise, and to practice you knoweldge of ROS**

You can look at a possible solution in `~/Desktop/RFLabs/lab3/lab3_pkg/src/`. This code makes Baxter move both arms.